It runs and moves in order from the top.
Settings include model changes.
>上から順に実行して動きます。
>設定はモデル変更などがあります。

Adapted from（参考元）: 

- https://github.com/camenduru/stable-diffusion-webui-colab/
- https://github.com/ddPn08/automatic1111-colab/
- https://github.com/TheLastBen/fast-stable-diffusion/

In [ ]:
#@title Clone webUI repository, etc.（webUIリポジトリのクローンなど）
!git clone https://github.com/thx-pw/stable-diffusion-webui --quiet
%cd /content/stable-diffusion-webui

!pip -qq install -U --pre triton
!pip -qq install xformers==0.0.17.dev449

!git clone https://github.com/thx-pw/sd_images_browser /content/stable-diffusion-webui/extensions/images-browser --quiet
!git clone https://github.com/thx-pw/a1111-sd-webui-tagcomplete /content/stable-diffusion-webui/extensions/a1111-sd-webui-tagcomplete --quiet
!git clone https://github.com/thx-pw/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/additional-networks --quiet
!git clone https://github.com/thx-pw/stable-diffusion-webui-aesthetic-gradients /content/stable-diffusion-webui/extensions/aesthetic-gradients --quiet
!git clone https://github.com/thx-pw/stable-diffusion-webui-pixelization /content/stable-diffusion-webui/extensions/pixelization --quiet
!git clone https://github.com/thx-pw/stable-diffusion-webui-instruct-pix2pix /content/stable-diffusion-webui/extensions/instruct-pix2pix --quiet
!git clone https://github.com/thx-pw/Hypernetwork-MonkeyPatch-Extension /content/stable-diffusion-webui/extensions/Hypernetwork-MonkeyPatch --quiet
!git clone https://github.com/thx-pw/sd-dynamic-thresholding /content/stable-diffusion-webui/extensions/dynamic-thresholding --quiet
!git clone https://github.com/thx-pw/sdweb-clip-changer /content/stable-diffusion-webui/extensions/clip-changer --quiet
!git clone https://github.com/thx-pw/ultimate-upscale-for-automatic1111 /content/stable-diffusion-webui/extensions/ultimate-upscale-for-automatic1111 --quiet
!git clone https://github.com/thx-pw/sd-webui-controlnet /content/stable-diffusion-webui/extensions/controlnet --quiet

# 今のバージョンだとエラー
# !git clone https://github.com/thx-pw/stable-diffusion-webui-two-shot /content/stable-diffusion-webui/extensions/stable-diffusion-webui-two-shot --quiet

# python 3.10じゃないとエラー
# !git clone https://github.com/thx-pw/kohya-sd-scripts-webui /content/stable-diffusion-webui/extensions/kohya-sd-scripts-webui --quiet

# Install Bore Tunnel https://github.com/AUTOMATIC1111/stable-diffusion-webui/issues/344
!wget -q https://github.com/ekzhang/bore/releases/download/v0.4.1/bore-v0.4.1-x86_64-unknown-linux-musl.tar.gz
!tar -xf bore-v0.4.1-x86_64-unknown-linux-musl.tar.gz
!rm -f bore-v0.4.1-x86_64-unknown-linux-musl.tar.gz
!cp bore /usr/bin/bore
!rm -rf bore

In [ ]:
#@title Download the model from Hugging Face.（Hugging Faceからモデルをダウンロード）
%cd /content/stable-diffusion-webui

import os
import io
import json
import requests
import pandas as pd

def resolve_raw_url(url):
  if url.startswith("https://huggingface.co/"):
    url = url.replace("/blob/", "/resolve/")
  elif url.startswith("https://github.com/"):
    url = url.replace("/blob/", "/raw/")
  return url

url = resolve_raw_url("https://github.com/thx-pw/stable-diffusion-webui-colab/blob/main/configs/model-config.csv")
response = requests.get(url, timeout=10)
models_df = pd.read_csv(io.StringIO(response.content.decode('utf-8')), header=0, index_col=0)

vaes_dict = {
  "None" : "",
  "trinart-derrida_final" : "https://huggingface.co/naclbit/trinart_derrida_characters_v2_stable_diffusion/blob/main/autoencoder_fix_kl-f8-trinart_characters.ckpt",
  "waifu-anime2" : "https://huggingface.co/hakurei/waifu-diffusion-v1-4/blob/main/vae/kl-f8-anime2.ckpt",
  "sd-vae-ft-mse" : "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/blob/main/vae-ft-mse-840000-ema-pruned.ckpt",
}

localizationes_dict = {
  "None" : "",
  "ar_AR" : "https://github.com/AUTOMATIC1111/stable-diffusion-webui-old-localizations",
  "de_DE" : "https://github.com/Strothis/stable-diffusion-webui-de_DE",
  "es_ES" : "https://github.com/innovaciones/stable-diffusion-webui-localization-es_ES",
  "fr-FR" : "https://github.com/AUTOMATIC1111/stable-diffusion-webui-old-localizations",
  "it_IT" : "https://github.com/Harvester62/stable-diffusion-webui-localization-it_IT",
  "ja_JP" : "https://github.com/Katsuyuki-Karasawa/stable-diffusion-webui-localization-ja_JP",
  "ko_KR" : "https://github.com/36DB/stable-diffusion-webui-localization-ko_KR",
  "no_NO" : "https://github.com/Cyanz83/stable-diffusion-webui-localization-no_NO",
  "pt_BR" : "https://github.com/M-art-ucci/stable-diffusion-webui-localization-pt_BR",
  "ru_RU" : "https://github.com/ProfaneServitor/stable-diffusion-webui-localization-ru_RU",
  "tr_TR" : "https://github.com/camenduru/stable-diffusion-webui-localization-tr_TR",
  "zh_CN" : "https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN",
  "zh_TW" : "https://github.com/benlisquare/stable-diffusion-webui-localization-zh_TW",
}

model = "waifu-v15" #@param ['None', 'waifu-v15', 'waifu-v14', 'Illuminati', 'th', 'DeDeDe', 'DeDeDeP', 'Counterfeit-V2.0', 'stable-v21-768', 'stable-v20-512-inpainting', 'stable-v21-512', 'stable-v20-512-depth', 'cool-japan', 'plat', 'tohoku-v2', 'mitsua-v10', 'waifu-v140-booru-14000', 'robo-v2', 'ACertainThing', 'ACertainModel', 'ACertainty', 'stable-v15', 'stable-v15-inpainting', '8528-final', 'waifu-v135-80000', 'waifu-v130-float16', 'sd-wikiart-v2', 'artstation-v1', 'cafe-instagram-unofficial', 'trinart-derrida_final', 'robo-v1', 'Counterfeit-V2.0']
vae = "waifu-anime2" #@param ['None', 'trinart-derrida_final', 'waifu-anime2', 'sd-vae-ft-mse']
localization = "ja_JP" #@param ['None', 'ar_AR', 'de_DE', 'es_ES', 'fr-FR', 'it_IT', 'ja_JP', 'ko_KR', 'no_NO', 'pt_BR', 'ru_RU', 'tr_TR', 'zh_CN', 'zh_TW']

if model != 'None':
  model_url = resolve_raw_url(models_df.at[model, 'url'])
  !wget {model_url} -P /content/stable-diffusion-webui/models/Stable-diffusion/

if vae != 'None':
  vae_url = resolve_raw_url(vaes_dict[vae])
  !wget {vae_url} -P /content/stable-diffusion-webui/models/VAE/

if localization != 'None':
  localization_url = localizationes_dict[localization]
  %cd /content/stable-diffusion-webui/extensions
  !git clone {localization_url} --quiet
  %cd /content/stable-diffusion-webui

styles_url = resolve_raw_url("https://github.com/thx-pw/stable-diffusion-webui-colab/blob/main/styles.csv")
!wget -q {styles_url} -O /content/stable-diffusion-webui/styles.csv

# モデルごとに設定を書き換え
if model != 'None':
  config_url = resolve_raw_url("https://github.com/thx-pw/stable-diffusion-webui-colab/blob/main/configs/config.json")
  response = requests.get(config_url, timeout=10)
  config_json = response.json()
  config_json["localization"] = localization
  config_json["CLIP_stop_at_last_layers"] = int(models_df.at[model, 'clip_stop_at_last_layers'])
  with open('/content/stable-diffusion-webui/config.json', "w", encoding="utf8") as file:
    json.dump(config_json, file, indent=4)

  ui_config_url = resolve_raw_url("https://github.com/thx-pw/stable-diffusion-webui-colab/blob/main/configs/ui-config.json")
  response = requests.get(ui_config_url, timeout=10)
  ui_config_json = response.json()
  ui_config_json["txt2img/Width/value"]  = int(models_df.at[model, 'width_value'])
  ui_config_json["txt2img/Height/value"] = int(models_df.at[model, 'height_value'])
  ui_config_json["img2img/Width/value"]  = int(models_df.at[model, 'width_value'])
  ui_config_json["img2img/Height/value"] = int(models_df.at[model, 'height_value'])
  ui_config_json["extras/Width/value"]   = int(models_df.at[model, 'width_value'])
  ui_config_json["extras/Height/value"]  = int(models_df.at[model, 'height_value'])
  with open('/content/stable-diffusion-webui/ui-config.json', "w", encoding="utf8") as file:
    json.dump(ui_config_json, file, indent=4)

def make_embedding_dir(model_name):
  positive_dir = os.path.join(textual_inversion_embeddings_colab, f"{model_name}/positive")
  negative_dir = os.path.join(textual_inversion_embeddings_colab, f"{model_name}/negative")
  os.makedirs(positive_dir, exist_ok=True)
  os.makedirs(negative_dir, exist_ok=True)
  return positive_dir, negative_dir

# embeddingのダウンロード、あとでリファクタリング
sd_webui_colab = "/content/stable-diffusion-webui"
textual_inversion_embeddings = "embeddings"
textual_inversion_embeddings_colab = os.path.join(sd_webui_colab, textual_inversion_embeddings)
if model == 'waifu-v15':
  positive_dir, negative_dir = make_embedding_dir(model)
  url = resolve_raw_url("https://huggingface.co/waifu-diffusion/wd-1-5-beta/blob/main/embeddings/wdgoodprompt.bin")
  !wget {url} -P {positive_dir}
  url = resolve_raw_url("https://huggingface.co/waifu-diffusion/wd-1-5-beta/blob/main/embeddings/wdbadprompt.pt")
  !wget {url} -P {negative_dir}
if model == 'Illuminati':
  positive_dir, negative_dir = make_embedding_dir(model)
  url = resolve_raw_url("https://huggingface.co/IlluminatiAI/Illuminati_Diffusion_v1.0/blob/main/embeds/positive/embellish1.pt")
  !wget {url} -P {positive_dir}
  url = resolve_raw_url("https://huggingface.co/IlluminatiAI/Illuminati_Diffusion_v1.0/blob/main/embeds/positive/embellish2.pt")
  !wget {url} -P {positive_dir}
  url = resolve_raw_url("https://huggingface.co/IlluminatiAI/Illuminati_Diffusion_v1.0/blob/main/embeds/positive/embellish3.pt")
  !wget {url} -P {positive_dir}
  url = resolve_raw_url("https://huggingface.co/IlluminatiAI/Illuminati_Diffusion_v1.0/blob/main/embeds/negative/nartfixer.pt")
  !wget {url} -P {negative_dir}
  url = resolve_raw_url("https://huggingface.co/IlluminatiAI/Illuminati_Diffusion_v1.0/blob/main/embeds/negative/nfixer.pt")
  !wget {url} -P {negative_dir}
  url = resolve_raw_url("https://huggingface.co/IlluminatiAI/Illuminati_Diffusion_v1.0/blob/main/embeds/negative/nfixernext.pt")
  !wget {url} -P {negative_dir}

In [ ]:
#@title Download the model from Google Drive.（Google Driveからモデルをダウンロード）
%cd /content/stable-diffusion-webui

import os
import glob
from google.colab import drive # type: ignore

use_google_drive = False #@param {type:"boolean"}
sd_webui_gdrive = "/content/drive/MyDrive/sd/stable-diffusion-webui" #@param {type:"string"}
sd_webui_colab = "/content/stable-diffusion-webui"

#@markdown Check the use_google_drive checkbox to copy the following files from the stable-diffusion-webui (sd_webui_gdrive) directory on Google Drive. Place models, etc. as follows.
#@markdown > use_google_driveにチェックを入れると、Googleドライブ上のstable-diffusion-webui(sd_webui_gdrive)ディレクトリから以下のファイルをコピーします。以下のようにモデルなどを配置してください。

#@markdown + Model
#@markdown  + models/Stable-diffusion/\*.ckpt
#@markdown  + models/Stable-diffusion/\*.safetensors
#@markdown  + models/Stable-diffusion/\*.yaml
#@markdown + VAE
#@markdown  + models/VAE/\*.ckpt
#@markdown  + models/VAE/\*.pt
#@markdown  + models/VAE/\*.safetensors
#@markdown  + models/Stable-diffusion/\*.vae.ckpt
#@markdown  + models/Stable-diffusion/\*.vae.pt
#@markdown  + models/Stable-diffusion/\*.vae.safetensors
#@markdown + Textual Inversion
#@markdown  + embeddings/\*.pt
#@markdown  + embeddings/\*.bin
#@markdown  + embeddings/\*.safetensors
#@markdown + Hypernetwork
#@markdown  + models/hypernetworks/*.pt
#@markdown  + models/hypernetworks/*.safetensors
#@markdown + Lora
#@markdown  + models/Lora/*.pt
#@markdown  + models/Lora/*.safetensors
#@markdown + Kohya Lora
#@markdown  + extensions/additional-networks/models/lora/*.pt
#@markdown  + extensions/additional-networks/models/lora/*.ckpt
#@markdown  + extensions/additional-networks/models/lora/*.safetensors
#@markdown + Aesthetic Embedding
#@markdown  + extensions/aesthetic-gradients/aesthetic_embeddings/*.pt
#@markdown  + extensions/aesthetic-gradients/aesthetic_embeddings/*.safetensors
#@markdown + Config
#@markdown  + styles.csv
#@markdown  + config.json
#@markdown  + ui-config.json

model_checkpoints = "models/Stable-diffusion"
model_model_vaes = "models/Stable-diffusion"
model_vae_vaes = "models/VAE"
textual_inversion_embeddings = "embeddings"
hypernetworks = "models/hypernetworks"
loras = "models/Lora"
kohya_loras = "extensions/additional-networks/models/lora/"
aesthetic_embeddings = "extensions/aesthetic-gradients/aesthetic_embeddings"
config_list = ["styles.csv", "config.json", "ui-config.json"]

model_checkpoints_gdrive = os.path.join(sd_webui_gdrive, model_checkpoints)
model_model_vaes_gdrive = os.path.join(sd_webui_gdrive, model_model_vaes)
model_vae_vaes_gdrive = os.path.join(sd_webui_gdrive, model_vae_vaes)
textual_inversion_embeddings_gdrive = os.path.join(sd_webui_gdrive, textual_inversion_embeddings)
hypernetworks_gdrive = os.path.join(sd_webui_gdrive, hypernetworks)
loras_gdrive = os.path.join(sd_webui_gdrive, loras)
kohya_loras_gdrive = os.path.join(sd_webui_gdrive, kohya_loras)
aesthetic_embeddings_gdrive = os.path.join(sd_webui_gdrive, aesthetic_embeddings)

model_checkpoints_colab = os.path.join(sd_webui_colab, model_checkpoints)
model_model_vaes_colab = os.path.join(sd_webui_colab, model_model_vaes)
model_vae_vaes_colab = os.path.join(sd_webui_colab, model_vae_vaes)
textual_inversion_embeddings_colab = os.path.join(sd_webui_colab, textual_inversion_embeddings)
hypernetworks_colab = os.path.join(sd_webui_colab, hypernetworks)
loras_colab = os.path.join(sd_webui_colab, loras)
kohya_loras_colab = os.path.join(sd_webui_colab, kohya_loras)
aesthetic_embeddings_colab = os.path.join(sd_webui_colab, aesthetic_embeddings)

if use_google_drive:
  drive.mount("/content/drive")

  # Google Driveにディレクトリを作成
  os.makedirs(model_checkpoints_gdrive, exist_ok=True)
  os.makedirs(model_model_vaes_gdrive, exist_ok=True)
  os.makedirs(model_vae_vaes_gdrive, exist_ok=True)
  os.makedirs(textual_inversion_embeddings_gdrive, exist_ok=True)
  os.makedirs(hypernetworks_gdrive, exist_ok=True)
  os.makedirs(loras_gdrive, exist_ok=True)
  os.makedirs(kohya_loras_gdrive, exist_ok=True)
  os.makedirs(aesthetic_embeddings_gdrive, exist_ok=True)

  # Google Colabにディレクトリを作成
  os.makedirs(hypernetworks_colab, exist_ok=True)
  os.makedirs(loras_colab, exist_ok=True)
  os.makedirs(kohya_loras_colab, exist_ok=True)
  os.makedirs(aesthetic_embeddings_colab, exist_ok=True)


  # 以下でファイルをコピー
  # モデルはコピーが遅いのでシンボリックリンクに変更

  # Model
  !ln -s {model_checkpoints_gdrive} "{model_checkpoints_colab}/link"
  # files = glob.glob(f"{model_checkpoints_gdrive}/*.ckpt")
  # files.extend(glob.glob(f"{model_checkpoints_gdrive}/*.safetensors"))
  # files.extend(glob.glob(f"{model_checkpoints_gdrive}/*.yaml"))
  # for model_checkpoint in files:
  #   !cp {model_checkpoint} {model_checkpoints_colab}

  # VAE
  files = glob.glob(f"{model_model_vaes_gdrive}/*.vae.ckpt")
  files.extend(glob.glob(f"{model_model_vaes_gdrive}/*.vae.pt"))
  files.extend(glob.glob(f"{model_model_vaes_gdrive}/*.vae.safetensors"))
  for model_vae in files:
    !cp {model_vae} {model_model_vaes_colab}
    
  # VAE
  files = glob.glob(f"{model_vae_vaes_gdrive}/*.ckpt")
  files.extend(glob.glob(f"{model_vae_vaes_gdrive}/*.pt"))
  files.extend(glob.glob(f"{model_vae_vaes_gdrive}/*.safetensors"))
  for model_vae in files:
    !cp {model_vae} {model_vae_vaes_colab}

  # Textual Inversion
  files = glob.glob(f"{textual_inversion_embeddings_gdrive}/*.pt")
  files.extend(glob.glob(f"{textual_inversion_embeddings_gdrive}/*.bin"))
  files.extend(glob.glob(f"{textual_inversion_embeddings_gdrive}/*.safetensors"))
  for textual_inversion_embedding in files:
    !cp {textual_inversion_embedding} {textual_inversion_embeddings_colab}

  # Hypernetwork
  files = glob.glob(f"{hypernetworks_gdrive}/*.pt")
  files.extend(glob.glob(f"{hypernetworks_gdrive}/*.safetensors"))
  for hypernetwork in files:
    !cp {hypernetwork} {hypernetworks_colab}
    
  # Lora
  files = glob.glob(f"{loras_gdrive}/*.pt")
  files.extend(glob.glob(f"{loras_gdrive}/*.safetensors"))
  for lora in files:
    !cp {lora} {loras_colab}

  # Kohya Lora
  files = glob.glob(f"{kohya_loras_gdrive}/*.pt")
  files.extend(glob.glob(f"{kohya_loras_gdrive}/*.ckpt"))
  files.extend(glob.glob(f"{kohya_loras_gdrive}/*.safetensors"))
  for lora in files:
    !cp {lora} {kohya_loras_colab}

  # Aesthetic Embedding
  files = glob.glob(f"{aesthetic_embeddings_gdrive}/*.pt")
  files.extend(glob.glob(f"{aesthetic_embeddings_gdrive}/*.safetensors"))
  for aesthetic_embedding in files:
    !cp {aesthetic_embedding} {aesthetic_embeddings_colab}

  # Config
  for config in config_list:
    config_gdrive = os.path.join(sd_webui_gdrive, config)
    if os.path.exists(config_gdrive):
      !cp {config_gdrive} {sd_webui_colab}


In [ ]:
#@title Launch the webUI.（webUIの起動）
%cd /content/stable-diffusion-webui

username = "user" #@param {type:"string"}
password = "pass" #@param {type:"string"}
tunnel = "gradio" #@param ["bore", "ngrok", "gradio"]
#@markdown get a token from for ngrok. https://dashboard.ngrok.com/get-started/your-authtoken （ngrokからトークンを取得）
ngrok_authtoken = "" #@param {type:"string"}
ngrok_region = "jp" #@param ["us", "eu", "au", "ap", "sa", "jp", "in"]


common_args = f"--xformers --disable-nan-check --lowram --enable-console-prompts --enable-insecure-extension-access --disable-safe-unpickle --update-check --gradio-auth {username}:{password}"
if tunnel == "bore":
  !STABLE_DIFFUSION_REPO=https://github.com/thx-pw/stablediffusion.git STABLE_DIFFUSION_COMMIT_HASH=72369d466203f9e4135d53faa085dbc272204607 ATTN_PRECISION=fp16 COMMANDLINE_ARGS="{common_args}" python launch.py & bore local 7860 --to bore.pub | sed -e "s|bore.pub|http://bore.pub|"
elif tunnel == "ngrok":
  !STABLE_DIFFUSION_REPO=https://github.com/thx-pw/stablediffusion.git STABLE_DIFFUSION_COMMIT_HASH=72369d466203f9e4135d53faa085dbc272204607 ATTN_PRECISION=fp16 COMMANDLINE_ARGS="{common_args} --ngrok {ngrok_authtoken} --ngrok-region {ngrok_region}" python launch.py
else:
  !STABLE_DIFFUSION_REPO=https://github.com/thx-pw/stablediffusion.git STABLE_DIFFUSION_COMMIT_HASH=72369d466203f9e4135d53faa085dbc272204607 ATTN_PRECISION=fp16 COMMANDLINE_ARGS="{common_args} --share" python launch.py


In [ ]:
#@title Save to zip file with same password as cell above. （上のセルと同じパスワードでzipファイルに保存）

%cd /content/stable-diffusion-webui
!zip -er --password={password} outputs.zip outputs 

In [ ]:
#@title Save the zip file to Google Drive. （Google Driveにzipファイルを保存）
from google.colab import drive 
drive_path = "/content/drive"
drive.mount(drive_path, force_remount=False)

!cp "/content/stable-diffusion-webui/outputs.zip" "/content/drive/MyDrive"